In [ ]:
import os
import numpy as np
import cv2
import yaml
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Custom YOLOv10 implementation or pre-trained weights loading
from yolov10 import YOLOv10Model  # Replace with actual YOLOv10 implementation


In [ ]:
import os
import cv2
import numpy as np
import yaml

def convert_annotations_to_yaml(class_dir, output_yaml):
    annotations = []
    
    for class_name in os.listdir(class_dir):
        class_path = os.path.join(class_dir, class_name)
        img_folder = os.path.join(class_path, 'img')  # Folder with image frames
        gt_file = os.path.join(class_path, 'groundtruth_rect.txt')
        
        if not os.path.exists(gt_file):
            continue
        
        with open(gt_file, 'r') as f:
            lines = f.readlines()
        
        objects = []
        for line in lines:
            bbox = list(map(float, line.strip().split()))
            objects.append({
                'bbox': bbox
            })
        
        for img_file in os.listdir(img_folder):
            if img_file.endswith('.jpg'):
                annotations.append({
                    'image': os.path.join(class_name, 'img', img_file),
                    'objects': objects
                })
    
    with open(output_yaml, 'w') as f:
        yaml.dump(annotations, f, default_flow_style=False)

# Usage
dataset_dir = 'path/to/your/dataset'
output_yaml_file = 'annotations.yaml'
convert_annotations_to_yaml(dataset_dir, output_yaml_file)


In [ ]:
def augment_data(image, label):
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.Rotate(limit=45, p=0.5),
        ToTensorV2()
    ])
    
    augmented = transform(image=image)
    return augmented['image'], label


In [ ]:
def load_data_from_yaml(yaml_file):
    with open(yaml_file, 'r') as f:
        data = yaml.load(f, Loader=yaml.SafeLoader)
    
    images = []
    labels = []
    
    for item in data:
        img_path = item['image']
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (416, 416))
        img = img / 255.0  # Normalize
        
        bbox = [obj['bbox'] for obj in item['objects']]
        labels.append(bbox)
        images.append(img)
    
    return np.array(images), np.array(labels)

# Load data
X, y = load_data_from_yaml(output_yaml_file)

# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define YOLOv10 model
model = YOLOv10Model(input_shape=(416, 416, 3), num_classes=32)  # Adjust num_classes as needed

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Training loop
history = model.fit(
    X_train, y_train, 
    validation_data=(X_val, y_val), 
    batch_size=16, epochs=50
)

# Evaluate the model
val_predictions = model.predict(X_val)
print(classification_report(np.argmax(y_val, axis=1), np.argmax(val_predictions, axis=1)))
print(confusion_matrix(np.argmax(y_val, axis=1), np.argmax(val_predictions, axis=1)))

# Save the model
model.save('coraltrack_yolov10_model.h5')

# Visualize some predictions
def visualize_predictions(images, predictions, true_labels):
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(10, 10))
    
    for i in range(9):
        plt.subplot(3, 3, i+1)
        plt.imshow(images[i])
        pred = np.argmax(predictions[i])
        true = np.argmax(true_labels[i])
        plt.title(f"Pred: {pred}, True: {true}")
        plt.axis('off')
    
    plt.show()

# Visualize predictions on validation data
visualize_predictions(X_val, val_predictions, y_val)
